In [1]:
import pandas as pd
import numpy as np

dfText = pd.read_csv('C:/Users/alire/OneDrive/data/statman_bitbucket/aki/LLM/March2024/all_text_columns.csv')
dfPatients = pd.read_csv(
    'C:/Users/alire/OneDrive/data/statman_bitbucket/aki/LLM/March2024/patients_for_python.csv'
)
dfCombined = pd.merge(
    dfPatients, dfText
    , on = ['project_id', 'operation_no']
    , how = 'inner'
).reset_index(drop = True)

X, y = dfCombined[['is_female', 'age', 'height_residual', 'bmi', 'optime', 'diagnosis', 'operation']], np.array(dfCombined['kdigo_stage'])
Xtext = dfCombined[['diagnosis', 'operation']]

In [2]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from TextEmbeddingFE.embed import Textcol2Mat
from TextEmbeddingFE.distill import Mat2ScoreClassifier
from sklearn.linear_model import LogisticRegression

C:\Users\alire\anaconda3\envs\devTEFE\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
textcol2Score = Pipeline([('textcol2Mat', Textcol2Mat(doc2vec_vector_size = 50)), ('mat2Score', Mat2ScoreClassifier(logit = True))])
colTrans = ColumnTransformer(
    [("textcol2Score", textcol2Score, ['diagnosis', 'operation'])]
    , remainder = 'passthrough'
)
pipe = Pipeline([('colTrans', colTrans), ('logit', LogisticRegression(penalty = None, max_iter = 100, solver = 'newton-cholesky'))])

pipe.fit(X, y)
pipe.predict_proba(X.iloc[:5, :])[:, 1]

array([0.15326826, 0.23625347, 0.17483044, 0.15209407, 0.20931785])

In [5]:
textcol2Score.fit(Xtext, y)
textcol2Score.predict_proba(Xtext.iloc[:5, :])

array([[0.25135766],
       [0.51438014],
       [0.25135766],
       [0.33965158],
       [0.25135766]])